# Calculate the percentage of anomaly
#### Jose Valles (jose.valles.leon@gmail.com)

### Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
import numpy as np
import calendar

sns.set()

from IPython.display import HTML

Import basin level code 2 and 3

In [2]:
BASIN_LEVEL3 = pd.read_csv(f'../output_modelo/cuenca_nivel3.csv',index_col="Codigo")
BASIN_LEVEL2 = pd.read_csv(f'../output_modelo/cuenca_nivel2.csv',index_col="Codigo")

In [3]:
def importmodelvariable(model_variable):
    df = pd.read_csv(f'../output_modelo/{model_variable}.csv')
    df = df.rename(columns={'-1': 'year','-1.1':'month'})
    df['date'] = pd.to_datetime(dict(year=df['year'],month=df['month'],day=1))
    df = df.set_index('date')
    df['days_in_month'] = df.index.days_in_month
    return df

def convertRunoff2Discharge(df_runoff):
    df_runoff_selected = df_runoff.drop(['year','month','days_in_month'],axis=1)
    df_discharge = pd.DataFrame(df_runoff_selected.values*1000*BASIN_LEVEL3.values,columns=df_runoff_selected.columns)
    df_discharge['days_in_month'] = df_runoff['days_in_month'].values
    df_discharge = df_discharge.loc[:, df_discharge.columns != 'days_in_month'].divide(df_discharge["days_in_month"]*24*3600, axis="index")
    df_discharge['date'] = df_runoff.index.values
    df_discharge = df_discharge.set_index('date')
    df_discharge['year'] = df_runoff['year'].values
    df_discharge['month'] = df_runoff['month'].values
    return df_discharge

def defineHydroSOScategory(VARIABLE_MENSUAL,VARIABLE_AVERAGE,VARIABLE):
    # create empty columns in the dataframe
    VARIABLE_MENSUAL['mean'] = np.nan
    VARIABLE_MENSUAL['average_percentage'] = np.nan
    VARIABLE_MENSUAL['rank_average'] = np.nan
    VARIABLE_MENSUAL['non_missing'] = np.nan


    for i in range(len(VARIABLE_MENSUAL)):
        # Extract the current month 
        m = VARIABLE_MENSUAL.month[i]
        # Extract the current year
        y = VARIABLE_MENSUAL.year[i]
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'rank_average']  = VARIABLE_MENSUAL.query('month==@m')[VARIABLE].rank()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'non_missing']  = VARIABLE_MENSUAL.query('month==@m')[VARIABLE].notnull().sum()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'mean'] = VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'average_percentage'] = (VARIABLE_MENSUAL[VARIABLE][i] - VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()) / VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()

    VARIABLE_MENSUAL['percentile'] = VARIABLE_MENSUAL['rank_average']/(VARIABLE_MENSUAL['non_missing']+1)
    criteria = [VARIABLE_MENSUAL['percentile'].between(0.90,1.00),
            VARIABLE_MENSUAL['percentile'].between(0.75,0.90),
            VARIABLE_MENSUAL['percentile'].between(0.25,0.75),
            VARIABLE_MENSUAL['percentile'].between(0.10,0.25),
            VARIABLE_MENSUAL['percentile'].between(0.00,0.10)]

    values = ['High flow','Above normal','Normal range','Below normal','Low flow']

    VARIABLE_MENSUAL['percentile_range'] = np.select(criteria,values,None)
    return VARIABLE_MENSUAL

In [4]:
hydrological_variable = ['Escorrentia_total','Escorrentia_sup','Escorrentia_sub','Pmedias','ETR','HumedadSuelo']

for hydro in hydrological_variable:
    if hydro == "Escorrentia_total":
        RUNOFF_total = importmodelvariable(hydro)
    elif hydro == "Pmedias":
        PRECIP = importmodelvariable(hydro)
    elif hydro == "ETR":
        ETR = importmodelvariable(hydro)
    elif hydro == "HumedadSuelo":
        SM = importmodelvariable(hydro)
    elif hydro == 'Escorrentia_sup':
        RUNOFF_sup = importmodelvariable(hydro)
    elif hydro == 'Escorrentia_sub':
        RUNOFF_sub = importmodelvariable(hydro)

In [5]:
HTML(PRECIP.tail(6).to_html(index=False))

year,month,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,days_in_month
2023,3,173.930,193.440,239.3300,240.0200,237.640,229.8700,167.580,141.780,141.120,131.930,140.100,140.590,154.680,161.860,153.110,144.070,113.450,105.520,108.420,119.770,135.940,136.840,117.770,117.150,107.920,112.590,114.550,118.420,161.110,192.400,202.570,207.390,190.260,205.5600,139.310,181.060,180.400,194.0300,192.850,194.560,187.910,181.320,129.190,139.200,162.000,167.060,139.250,161.770,168.670,166.690,169.920,172.120,161.330,158.590,172.470,149.780,115.8100,92.0230,93.8960,85.7070,59.5190,66.6720,57.8970,66.464,54.833,54.6070,44.1830,47.2600,44.0490,45.1070,55.5830,47.8740,40.7760,36.6150,40.751,40.8060,41.1310,41.8470,99.6920,96.4710,98.463,99.7060,89.6380,74.9690,54.1990,55.4580,51.9680,41.678,48.5450,60.8190,80.0500,101.2000,102.120,85.1250,91.4130,92.0180,92.1950,79.4850,74.5270,73.3480,77.0470,83.3800,79.5010,81.5210,79.3840,79.2270,79.1430,79.0140,124.640,109.9800,107.0100,110.750,122.890,78.2120,83.450,101.0500,93.9930,81.749,95.2260,104.6100,124.790,127.030,134.860,135.720,136.660,147.460,137.900,137.730,138.790,145.370,141.490,139.970,139.760,146.980,150.250,177.380,180.700,182.030,200.960,208.010,145.020,176.800,139.500,140.930,144.770,146.730,146.410,150.610,151.470,153.160,157.920,161.830,183.260,183.110,174.570,172.090,184.150,123.580,147.490,123.000,112.350,99.345,155.270,131.610,178.610,188.250,134.120,154.890,136.380,155.020,162.660,165.120,164.890,181.570,190.070,172.960,197.710,195.300,189.590,166.510,199.360,206.370,206.730,205.860,111.460,110.090,114.500,128.710,137.900,148.060,167.350,187.660,150.460,146.020,145.780,144.840,143.200,140.700,135.790,126.210,136.870,123.900,124.210,125.980,125.880,127.620,125.080,136.650,134.320,145.000,122.680,140.380,157.160,141.560,128.200,150.750,138.220,142.350,138.870,156.490,128.190,127.410,161.880,116.320,159.790,130.150,136.820,140.780,143.200,156.950,156.720,109.320,143.560,174.430,178.120,197.700,216.860,196.200,196.900,172.370,138.850,160.910,111.5400,139.180,153.700,151.660,180.310,137.910,140.590,172.870,147.000,132.460,117.6700,125.3700,145.740,138.670,130.900,103.5100,104.3500,122.0400,106.2900,109.9300,96.9340,100.6100,172.700,150.660,158.2000,150.3600,109.4700,119.3900,106.5400,97.5780,109.8000,94.2580,93.966,96.7250,95.5910,138.2400,138.6600,141.2600,131.5100,131.1100,134.400,120.9700,98.0320,99.9570,95.0090,96.9400,94.2750,96.9550,92.7210,94.5900,85.463,92.8300,90.3960,84.2960,31
2023,4,22.405,13.270,7.5846,9.6145,10.869,9.0914,17.814,50.531,56.080,59.843,59.254,58.552,56.538,55.481,58.713,55.568,14.377,19.444,25.096,37.420,32.608,24.669,24.808,26.084,25.358,33.008,41.669,42.442,58.032,67.919,73.476,68.332,66.064,87.7510,57.652,80.693,83.536,103.5800,95.118,90.279,77.355,65.849,46.381,51.367,45.537,41.411,48.818,52.077,39.366,37.221,49.861,52.725,36.679,32.320,33.

### Convert from runoff to discharge

In [6]:
DISCHARGE = convertRunoff2Discharge(RUNOFF_total)

In [7]:
HTML(DISCHARGE.tail(6).to_html(index=True))

,101,102,103,105,106,107,108,109,110,111,112,114,115,116,117,119,120,123,125,128,130,131,132,133,134,135,136,137,138,139,140,142,146,148,150,155,158,160,163,165,167,168,170,171,172,173,174,175,176,177,178,179,180,183,186,189,190,193,196,199,201,204,208,210,211,212,213,214,215,216,217,220,221,222,223,224,225,226,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,260,262,264,266,268,270,274,275,276,277,279,280,281,282,283,284,285,286,287,288,289,290,293,294,295,297,298,300,301,305,306,310,312,315,316,318,320,325,330,333,334,335,338,406,407,410,411,412,413,414,415,416,417,418,419,420,422,424,426,428,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,450,452,453,454,455,457,458,459,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,522,524,526,530,532,534,536,538,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,575,580,581,582,583,584,585,586,587,588,589,600,601,602,603,604,605,606,607,608,609,610,611,612,613,615,616,618,620,625,630,635,640,645,650,651,652,653,654,655,656,657,658,660,661,662,666,670,673,677,680,683,687,year,month
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-01,6.648432,3.842178,13.550911,4.103038,4.792139,18.846121,9.780653,0.287414,0.022268,0.019670,0.005339,0.023905,0.079888,0.029240,0.128971,1.775248,0.149306,0.094660,0.687731,0.081265,0.340360,0.409754,0.084755,0.170926,2.146315,0.754091,0.110217,0.180710,2.344049,1.222950,0.019097,2.927017,3.485036,1.958887,3.498625,5.956983,6.175922,0.422352,0.892097,1.675090,4.477731,0.397120,0.318056,14.434203,0.652711,0.601013,6.554637,1.527413,0.877007,0.736751,1.421668,0.456302,0.001573,0.017016,1.484741,0.024720,2.373238e-07,0.000108,8.120789e-07,5.012927e-07,3.622312e-09,2.765393e-07,4.004840e-07,6.991637e-07,3.753138e-07,7.507426e-08,5.802509e-08,5.238053e-08,1.245225e-07,1.259380e-07,2.404146e-07,2.839830e-09,1.152613e-07,9.586492e-08,1.431259e-07,1.202449e-08,3.515700e-08,2.331929e-07,1.312572e-06,4.043291e-07,2.392130e-06,1.615833e-06,3.175553e-06,3.304516e-06,1.084640e-07,1.792208e-06,6.833468e-08,3.467977e-08,1.096203e-08,6.751445e-08,2.624414e-06,4.532784e-06,1.812697e-06,1.668263e-07,5.246629e-07,3.137079e-07,2.316487e-07,0.000092,0.000148,0.000326,0.000387,0.003136,3.803241e-05,0.007695,0.000680,0.000242,0.000129,0.000226,0.068511,0.001192,0.002381,0.035544,0.672981,0.002003,0.000319,0.001409,0.000977,0.001321,0.000298,0.000975,0.000755,1.365783,0.006658,0.001871,0.000343,4.403719,1.554735,0.006762,0.001387,3.565618,1.007646,0.003293,1.777397,0.009812,1.482396,0.077282,0.757246,6.851461,0.308992,0.327254,6.964855,0.414054,0.288316,0.068761,0.127651,0.818601,2.817445,0.531912,2.489831,4.343058,0.432323,2.318557,0.066179,0.745706,0.962141,2.646750,0.833515,0.435935,1.570833,0.183724,0.234946,0.230653,8.067151,5.748777,16.910902,1.070923,1.827427,4.050609,3.151039,7.673131,5.088244,20.860377,2.836558,4.586097,0.185717,20.104115,0.427674,3.383100,1.656413,5.262605,0.677380,0.209238,0.005362,1.109586,0.027597,0.086840,0.050982,0.145062,0.081403,0.537320,5.345005,5.102975,6.091609,0.011012,0.009914,0.030134,0.343392,0.590442,0.341771,0.080548,0.345799,0.012815,0.051542,0.088855,0.090851,0.134146,0.101239,0.005078,0.042883,0.042389,0.257092,0.180003,0.017951,0.053635,1.210587,0.038664,0.040610,0.087348,3.426397,1.065154,0.129253,0.050820,3.121360,0.005679,0.799773,0.115164,0.051001,0.046115,0.062070,1.265025,1.029702,0.882961,3.594682,1.872939,13.676538,6.965336,12.026317,14.260302,12.565700,4.536616,0.862637,0.115221,0.000182,1.322883,0.009647,5.878778,1.369147,0.047776,0.000356,0.990416,0.000317,0.000182,2.332382,0.000059,1.193613,2.132730,0.847561,0.0

### Pleasee select the runoff type for the analysis (RUNOFF_total, RUNOFF_sub, RUNOFF_sup)

In [8]:
# Select the runoff type 
RUNOFF = RUNOFF_total

### Select reference period from 'year_start' to 'year_end'

Select variable of interest

In [9]:
year_start = 1981
year_end = 2010
# Caudal
SELECTED_REF_DISCHARGE= DISCHARGE[(DISCHARGE['year'] >= year_start) & (DISCHARGE['year'] <= year_end)]
# Escorrentia tota
SELECTED_REF_RUNOFF = RUNOFF[(RUNOFF['year'] >= year_start) & (RUNOFF['year'] <= year_end)]
# Precip
SELECTED_REF_PRECIP = PRECIP[(PRECIP['year'] >= year_start) & (PRECIP['year'] <= year_end)]

## Calculate the percentage of anomaly for each basin code level 2 and a specific month and year

In [10]:
#--
month_analyis = 8
year_analysis = 2023

AVERAGE_PERCENTAGE = pd.DataFrame()
AVERAGE_PERCENTAGE['codigo'] = BASIN_LEVEL2.columns
AVERAGE_PERCENTAGE['discharge'] = np.nan
AVERAGE_PERCENTAGE['runoff'] = np.nan
AVERAGE_PERCENTAGE['precip'] = np.nan



for basin in BASIN_LEVEL2.columns:
    # filter the basin code bas
    filter_col = [col for col in DISCHARGE if col.startswith(str(basin))]
    # Select basins level 3 that belongs to level 2 basin
    # Discharge
    DISCHARGE_FILTER = DISCHARGE[filter_col]
    DISCHARGE_SELECTED = pd.DataFrame()
    DISCHARGE_SELECTED['year'] = DISCHARGE['year']
    DISCHARGE_SELECTED['month'] = DISCHARGE['month']    
    DISCHARGE_SELECTED['discharge'] = DISCHARGE_FILTER.sum(axis=1)
    DISCHARGE_REF = DISCHARGE_SELECTED[(DISCHARGE_SELECTED['year'] >= year_start) & (DISCHARGE_SELECTED['year'] <= year_end)]
    # Runoff
    RUNOFF_FILTER = RUNOFF[filter_col]
    RUNOFF_SELECTED = pd.DataFrame()
    RUNOFF_SELECTED['year'] = RUNOFF['year']
    RUNOFF_SELECTED['month'] = RUNOFF['month']    
    RUNOFF_SELECTED['runoff'] = RUNOFF_FILTER.mean(axis=1)
    RUNOFF_REF = RUNOFF_SELECTED[(RUNOFF_SELECTED['year'] >= year_start) & (RUNOFF_SELECTED['year'] <= year_end)]
    # Precip
    PRECIP_FILTER = PRECIP[filter_col]
    PRECIP_SELECTED = pd.DataFrame()
    PRECIP_SELECTED['year'] = PRECIP['year']
    PRECIP_SELECTED['month'] = PRECIP['month']    
    PRECIP_SELECTED['precip'] = PRECIP_FILTER.mean(axis=1)
    PRECIP_REF = PRECIP_SELECTED[(PRECIP_SELECTED['year'] >= year_start) & (PRECIP_SELECTED['year'] <= year_end)]
    # CALCULATE CLIMATOLOGICAL NORMALS
    # Discharge
    DISCHARGE_AVERAGE = DISCHARGE_REF.groupby(DISCHARGE_REF.month).mean()
    # Runoff
    RUNOFF_AVERAGE = RUNOFF_REF.groupby(RUNOFF_REF.month).mean()
    # Precip
    PRECIP_AVERAGE = PRECIP_REF.groupby(PRECIP_REF.month).mean()
    # HydroSOS Category
    hydroSOS = defineHydroSOScategory(RUNOFF_SELECTED,RUNOFF_AVERAGE,'runoff')
    # Calculate the anomaly
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'discharge'] = (DISCHARGE_SELECTED.query('month == @month_analyis & year == @year_analysis')['discharge'].item() - DISCHARGE_AVERAGE.query('month == @month_analyis')['discharge'].item()) / DISCHARGE_AVERAGE.query('month == @month_analyis')['discharge'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'runoff'] = (RUNOFF_SELECTED.query('month == @month_analyis & year == @year_analysis')['runoff'].item()- RUNOFF_AVERAGE.query('month == @month_analyis')['runoff'].item()) / RUNOFF_AVERAGE.query('month == @month_analyis')['runoff'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'precip'] = (PRECIP_SELECTED.query('month == @month_analyis & year == @year_analysis')['precip'].item() - PRECIP_AVERAGE.query('month == @month_analyis')['precip'].item()) / PRECIP_AVERAGE.query('month == @month_analyis')['precip'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'precip_val'] = PRECIP_SELECTED.query('month == @month_analyis & year == @year_analysis')['precip'].item()
    AVERAGE_PERCENTAGE.loc[AVERAGE_PERCENTAGE.eval('codigo==@basin'),'hydroSOS_category'] = hydroSOS.query('month == @month_analyis & year == @year_analysis')['percentile_range'].item()

In [11]:
HTML(AVERAGE_PERCENTAGE.tail(12).to_html(index=False))

codigo,discharge,runoff,precip,precip_val,hydroSOS_category
56,-0.375881,-0.356820,-0.020823,85.929000,Normal range
57,-0.913898,-0.673302,-0.304824,49.746667,Below normal
58,-0.646977,-0.810695,-0.448688,35.942800,Below normal
60,-0.682218,-0.696105,0.188413,114.228000,Below normal
61,-0.574902,-0.550878,0.219116,111.086286,Below normal
62,-0.628907,-0.643151,0.606011,128.575000,Normal range
63,-0.250843,-0.259632,1.061999,157.985000,Normal range
64,-0.381160,-0.375072,1.159707,161.370000,Normal range
65,-0.829341,-0.843376,0.262258,96.174778,Below normal
66,-0.491721,-0.493428,1.217766,164.377500,Normal range


In [ ]:
AVERAGE_PERCENTAGE.to_clipboard(index=False)

## Calculate the percentage of anomaly for each basin code level 2 and a specific year

In [ ]:
#--
year_analysis = 2019

AVERAGE_PERCENTAGE_YEAR = pd.DataFrame()
AVERAGE_PERCENTAGE_YEAR['codigo'] = BASIN_LEVEL2.columns
AVERAGE_PERCENTAGE_YEAR['runoff'] = np.nan
AVERAGE_PERCENTAGE_YEAR['runoff_anomaly'] = np.nan
AVERAGE_PERCENTAGE_YEAR['runoff_mean'] = np.nan



for basin in BASIN_LEVEL2.columns:
    # filter the basin code bas
    filter_col = [col for col in DISCHARGE if col.startswith(str(basin))]
    # Select basins level 3 that belongs to level 2 basin
    # Runoff
    RUNOFF_FILTER = RUNOFF[filter_col]
    RUNOFF_SELECTED = pd.DataFrame()
    RUNOFF_SELECTED['year'] = RUNOFF['year']
    RUNOFF_SELECTED['month'] = RUNOFF['month']    
    RUNOFF_SELECTED['runoff'] = RUNOFF_FILTER.mean(axis=1)
    RUNOFF_REF = RUNOFF_SELECTED[(RUNOFF_SELECTED['year'] >= year_start) & (RUNOFF_SELECTED['year'] <= year_end)]

    sum_avg_runoff_year = RUNOFF_SELECTED.groupby(['year'])['runoff'].sum()
    sum_avg_runoff_year = sum_avg_runoff_year.to_frame()

    # Runoff
    RUNOFF_AVERAGE = RUNOFF_REF.groupby(['year'])['runoff'].sum()
    RUNOFF_AVERAGE = RUNOFF_AVERAGE.to_frame()

    # Calculate the anomaly
    AVERAGE_PERCENTAGE_YEAR.loc[AVERAGE_PERCENTAGE_YEAR.eval('codigo==@basin'),'runoff'] = sum_avg_runoff_year.query('year == @year_analysis')['runoff'].item()
    AVERAGE_PERCENTAGE_YEAR.loc[AVERAGE_PERCENTAGE_YEAR.eval('codigo==@basin'),'runoff_anomaly'] = (sum_avg_runoff_year.query('year == @year_analysis')['runoff'].item() - RUNOFF_AVERAGE['runoff'].mean()) / RUNOFF_AVERAGE['runoff'].mean()
    AVERAGE_PERCENTAGE_YEAR.loc[AVERAGE_PERCENTAGE_YEAR.eval('codigo==@basin'),'runoff_mean'] = RUNOFF_AVERAGE['runoff'].mean()

In [ ]:
HTML(AVERAGE_PERCENTAGE_YEAR.head(6).to_html(index=False))